In [70]:
import numpy as np
import cv2 as cv
from scipy.ndimage import label


img = cv.imread("segmentation.jpg")
img = cv.resize(img, (800, 800))
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# noise removal
ret, img_morph = cv.threshold(gray, 0, 255, cv.THRESH_OTSU)
kernel = np.ones((3, 3), np.uint8)
img_morph = cv.morphologyEx(img_morph, cv.MORPH_OPEN, kernel)
img_copy = img.copy()

In [71]:
low_threshold = 0
high_threshold = 255
iteration_num = 5
INTERNAL_STEPS = False


def update():
	# sure background area
	dilated_img = cv.dilate(img_morph, kernel, iterations=iteration_num)

	# erosion
	dilated_img = dilated_img - cv.erode(dilated_img, None)

	# Finding sure foreground area
	distances_transform = cv.distanceTransform(img_morph, cv.DIST_L2, 3)

	if INTERNAL_STEPS:
		cv.imshow('distances', distances_transform)

	_, thresh = cv.threshold(distances_transform, low_threshold, high_threshold, cv.THRESH_BINARY)

	if INTERNAL_STEPS:
		# show thresh changing
		cv.imshow('thresh', thresh)

	labels, num_features = label(thresh)
	# Multiplying label array on 255 / (number of features + 1) to prevent deviding on zero
	labels = labels * (255 / (num_features + 1))
	# Completing the markers now
	labels[dilated_img == 255] = 255
	# Using watershed
	labels = labels.astype(np.int32)
	labels = cv.watershed(img, labels)
	if INTERNAL_STEPS:
		cv.imshow('labels', cv.convertScaleAbs(labels))
	# Changing values that is equal -1
	labels[labels == -1] = 0
	# Getting outlines as result
	markers = 255 - labels.astype(np.uint8)
	markers[markers != 255] = 0
	markers = cv.dilate(markers, None)
	#copy original image to add on it edges
	result = img.copy()
	result[markers == 255] = (0, 255, 0)
	if INTERNAL_STEPS:
		cv.imshow('Outlines', markers)
	cv.imshow('Result', result)


def change_low_threshold(val):
	global low_threshold
	low_threshold = val
	update()


def change_iteration_num(val):
	global iteration_num
	iteration_num = val
	update()

In [72]:
cv.imshow('thresh', img_morph)
cv.createTrackbar('thresh low', 'thresh', 20, 255, change_low_threshold)
cv.createTrackbar('iterations', 'thresh', 5, 100, change_iteration_num)
cv.waitKey(0)
cv.destroyAllWindows()